# Easy Agent Tutorial
This notebook file provide three examples of using LLM based agents with different tool sets.

prequisites:
- Python 3.10+
- Install required packages:
  ```bash
  pip install -r "mcp[cli]" smolagent
  ```

## Task Description

如README所述，该项目应用三种方案，从不同的角度实现了agentic RAG的功能。为了演示，这一次我们将会构建一个信安四大会的查询，来进行感兴趣论文的搜索以及基于题目选择合适的会议进行投稿。

## Data Preparation

In [1]:
# 我们将使用dblp数据集来进行演示。首先下载四大会最近几年的会议论文数据：
import requests
import json
import os

if not os.path.exists("dblp_sec_papers.json"):
    with requests.Session() as sess:
        url = "https://dblp.org/search/publ/api"
        params = {"q": "security", "format": "json"}
        tocs = {
            "ndss": "toc:db/conf/ndss/ndss2025.bht:",
            "sp": "toc:db/conf/sp/sp2025.bht:",
            "ccs": "toc:db/conf/ccs/ccs2025.bht:",
            "usenix": "toc:db/conf/uss/uss2025.bht:",
        }
        papers = []
        for k, v in tocs.items():
            response = sess.get(url, params={"q": v, "h": 1000, "format": "json"})
            data = response.json()
            data = data["result"]["hits"]["hit"]
            papers.extend(data)
        with open(f"dblp_sec_papers.json", "w", encoding="utf-8") as f:
            json.dump(papers, f, ensure_ascii=False, indent=4)
else:
    with open(f"dblp_sec_papers.json", "r", encoding="utf-8") as f:
        papers = json.load(f)
papers = [x["info"] for x in papers]
titles = [f"[{x['venue']} {x['year']}] {x['title']}" for x in papers]

In [5]:
# 创建并保存向量数据库
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()
if "db" not in globals():
    db = FAISS.from_texts(titles, OpenAIEmbeddings())
db.save_local("faiss_db")

在开始之前，先看一下传统的工具调用大概长啥样，有怎样的优缺点

In [7]:
import json
import logging
import re

import dotenv
import tenacity
import yaml
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
from openai.types import *
from openai.types.chat import *


def raw_toolcall():
    client = OpenAI()
    db = FAISS.load_local(
        "faiss_db",
        OpenAIEmbeddings(),
        allow_dangerous_deserialization=True,
    )
    tools_def = [
        {
            "type": "function",
            "name": "query_db",
            "description": "accept keyword and return related information",
            "parameters": {
                "type": "object",
                "properties": {
                    "kw": {
                        "type": "string",
                        "description": "The keyword to query the database",
                    }
                },
                "required": ["kw"],
            },
        }
    ]

    def real_ask(question: str):
        from openai.types.responses.response_input_param import Message

        input_msgs: list[Message] = [
            {
                "type": "message",
                "role": "system",
                "content": f"You are a helpful research assistant. When given a question, you must first decide if you need to query the database to get relevant information. If so, use the tool 'query_db' with appropriate keywords extracted from the question. After getting the information, provide a comprehensive answer based on both the retrieved information and your own knowledge.",
            },
            {
                "type": "message",
                "role": "user",
                "content": question,
            },
        ]
        resp = client.responses.create(
            model="gpt-5.1",
            tools=tools_def,
            input=input_msgs,
            tool_choice="required",
        )
        print(resp.output)
        for toolcall in resp.output:
            if toolcall.type != "function_call":
                continue
            if toolcall.name == "query_db":
                kw = json.loads(toolcall.arguments)["kw"]
                print(f"Keyword: {kw}")
                docs = []
                for skw in kw.split():
                    if not (skw := skw.strip()):
                        continue
                    print(f"Searching for keyword: {skw}")
                    docs.extend(db.similarity_search(skw, k=30))
                rag_result = "\n".join([doc.page_content for doc in docs])
                input_msgs.append(toolcall)
                input_msgs.append(
                    {
                        "type": "function_call_output",
                        "call_id": toolcall.call_id,
                        "output": str(rag_result),
                    }
                )
                print(f"{kw=} appended.")
        if input_msgs[-1]["type"] == "function_call_output":
            resp = client.responses.create(
                model="gpt-5.1",
                input=input_msgs,
                # tools=tools_def,
                stream=True,
            )
            for chunk in resp:
                if chunk.type == "response.output_text.delta":
                    print(chunk.delta, end="", flush=True)
            print()
        else:
            print(resp.output_text)

    while True:
        inp = input("=> ")
        if not inp.strip():
            break
        real_ask(inp)


raw_toolcall()

[ResponseFunctionToolCall(arguments='{"kw":"blockchain LLM papers"}', call_id='call_ZNk69x2z6uSxuPEyhldVmhRc', name='query_db', type='function_call', id='fc_0edcc91604d6e0a4006938e28cf5c0819282e662d761ad49ea', status='completed')]
Keyword: blockchain LLM papers
Searching for keyword: blockchain
Searching for keyword: LLM
Searching for keyword: papers
kw='blockchain LLM papers' appended.
你这个方向目前还比较前沿，严格意义上“区块链 × LLM”的论文不算多，但已经有几类比较典型的结合方式，可以按“区块链为LLM赋能”和“LLM为区块链赋能”来找文献。我先列代表性论文和关键词，方便你自己去搜（Google Scholar / arXiv / dblp），再给你一个按方向分类的阅读建议。

下面所有英文标题你直接复制去搜索就能找到 PDF。

---

## 一、LLM 为区块链赋能（用 LLM 做智能合约/链上安全/分析）

### 1. 智能合约分析与形式化验证

- **PropertyGPT: LLM-driven Formal Verification of Smart Contracts through Retrieval-Augmented Property Generation**  
  NDSS 2025（网络与分布式系统安全研讨会）  
  关键词：  
  - “LLM-driven Formal Verification of Smart Contracts”  
  - “Retrieval-Augmented Property Generation”  
  核心思路：用 LLM 自动生成合约的安全属性 / 规范，再结合形式化验证工具检查。适合关注“LLM + 智能合约安全”的同学。

- 可一并检索的关键词：  
  - “LLM for smart co

### 传统工具调用的优缺点

- 优点：直接用模型原生的 function/tool 调用协议，链路短、开销低，JSON Schema 参数校验清晰。
- 优点：可以精确控制何时调用工具、使用 `tool_choice` 等参数强制执行，消息格式透明、便于调试和流式输出。
- 优点：依赖少，不绑框架，易于插入到现有服务或与其他编排层组合。
- 缺点：需要手写对话状态管理、工具输入输出拼接，容易出错且样板代码多。
- 缺点：缺少自动规划/多步推理、重试、fallback 等封装能力，复杂流程要自行实现。
- 缺点：与特定模型/协议耦合，换提供方或多模型时需适配；安全性与数据清洗（如反序列化、去重）也要自管。

## SmolAgent::CodeAgent

In [5]:
import yaml
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from smolagents import CodeAgent, OpenAIServerModel, tool

load_dotenv()

db = FAISS.load_local(
    "faiss_db",
    OpenAIEmbeddings(),
    allow_dangerous_deserialization=True,
)
content = f"You are a helpful research assistant. When given a question, you must first decide if you need to query the database to get relevant information. If so, use the tool 'query_db' with appropriate keywords extracted from the question. After getting the information, provide a comprehensive answer based on both the retrieved information and your own knowledge."


@tool
def query_db(kw: str) -> str:
    """
    accept keyword and return related paper titles

    Args:
        kw (str): The keyword to query the database
    """
    docs = []
    for skw in kw.split():
        if not (skw := skw.strip()):
            continue
        print(f"Searching for keyword: {skw}")
        docs.extend(db.similarity_search(skw, k=3))
    rag_result = "\n".join([doc.page_content for doc in docs])
    return rag_result


agent = CodeAgent(
    model=OpenAIServerModel("gpt-5.1"),
    tools=[query_db],
    verbosity_level=1,
    stream_outputs=True,
)

# from Gradio_UI import GradioUI

# GradioUI(agent).launch()

agent.run(f"{content}\n\n{input("=> ")}")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are a helpful research assistant. When given a question, you must first decide if you need to query the     │
│ database to get relevant information. If so, use the tool 'query_db' with appropriate keywords extracted from   │
│ the question. After getting the information, provide a comprehensive answer based on both the retrieved         │
│ information and your own knowledge.                                                                             │
│                                                                                                                 │
│ 精选几篇llm安全的论文                                                                                           │
│                                                                                                                 │
╰─ OpenAIModel - gpt-5.1 ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output()

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Thought:                                                                                                       
  # The user asks (in Chinese) to "select a few LLM safety papers".                                                
  # I should query the database using keywords like "LLM safety", "AI alignment", "AI safety", etc.,               
  # then use my own knowledge plus the returned titles to recommend and briefly describe several key papers.       
  # Step 1: query the DB for relevant titles.                                                                      
                                                                                                                   
  kw = "large language model safety alignment robustness red teaming RLHF"                                         
  titles = query_db(kw=kw)                                                                                         
  print(titles)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Searching for keyword: large
Searching for keyword: language
Searching for keyword: language
Searching for keyword: model
Searching for keyword: model
Searching for keyword: safety
Searching for keyword: safety
Searching for keyword: alignment
Searching for keyword: alignment
Searching for keyword: robustness
Searching for keyword: robustness
Searching for keyword: red
Searching for keyword: red
Searching for keyword: teaming
Searching for keyword: teaming
Searching for keyword: RLHF
Searching for keyword: RLHF


...

Output()

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # 这里不再导入 final_answer，直接在逻辑说明后用工具调用即可。                                                    
  text = """我从你给出的 NDSS / Oakland (SP) / USENIX Security / CCS 2025                                          
  论文列表里，挑出几篇和“大语言模型安全”直接相关、很值得关注的工作，按主题简单分组，方便你做调研或写               
  survey。下面都是你日志里已经出现的论文，并结合题目+常见研究方向做简要解读。                                      
                                                                                                                   
  ---                                                                                                              
                                                                                                                   
  ## 一、越狱攻击与越狱防御（Jailbreak & Prompt 攻击）                                                             
                                                                                                                   
  ### 1. JBShield: Defending Large Language Models from Jailbreak Attacks through Activated Concept Analysis and   
  Manipulation                                                                                                     
  **会议**：USENIX Security 2025                                                                                   
                                                                                                                   
  **关键词**：越狱防御、内部激活/概念分析、安全对齐保持。                                                          
  **可能内容**：                                                                                                   
  - 利用“激活概念分析”（activated concept                                                                          
  analysis）来理解模型内部表征哪些“危险概念”（如暴力、仇恨等），在检测到这些概念被触发时进行抑制或重写，从而阻止   
  越狱提示成功。                                                                                                   
  - 属于“模型内部机制层面”的防御，而不仅是简单规则过滤。                                                           
                                                                                                                   
  **为什么值得看**：                                                                                               
  - 越狱防御是 LLM 安全的核心问题之一，这篇代表了一种“从内部表征出发”的防御路线，可以和基于 RLHF /                 
  审查规则的办法做对比。                                                                                           
                                                                                                                   
  ---                                                                                                              
                                                                                                                   
  ### 2. TwinBreak: Jailbreaking LLM Security Alignments based on Twin Prompts                                     
  **会议**：USENIX Security 2025                                                                                   
                                                                                                                   
  **关键词**：越狱攻击、提示结构设计、对齐绕过。                                                                   
  **可能内容**：                                                                                                   
  - 提出一种 “Twin Prompts（双提示）”                                                                              
  结构，通过构造两段关联提示，让模型在遵守一段指令的同时无意间违反另一段更高层次的安全对齐约束。                   
  - 系统化研究这种结构是否对主流对齐策略（RLHF、system prompt 等）具有普适威力。                                   
                                                                                                                   
  **为什么值得看**：                                                                                               
  - 如果你在研究越狱攻击方法，这篇可能给出了一套新的攻击范式与评测基准。                                           
  - 也可作为防御方理解攻击模式的素材。                                   

Final answer: 我从你给出的 NDSS / Oakland (SP) / USENIX Security / CCS 2025 
论文列表里，挑出几篇和“大语言模型安全”直接相关、很值得关注的工作，按主题简单分组，方便你做调研或写 
survey。下面都是你日志里已经出现的论文，并结合题目+常见研究方向做简要解读。

---

## 一、越狱攻击与越狱防御（Jailbreak & Prompt 攻击）

### 1. JBShield: Defending Large Language Models from Jailbreak Attacks through Activated Concept Analysis and 
Manipulation  
**会议**：USENIX Security 2025  

**关键词**：越狱防御、内部激活/概念分析、安全对齐保持。  
**可能内容**：
- 利用“激活概念分析”（activated concept 
analysis）来理解模型内部表征哪些“危险概念”（如暴力、仇恨等），在检测到这些概念被触发时进行抑制或重写，从而阻止越狱
提示成功。
- 属于“模型内部机制层面”的防御，而不仅是简单规则过滤。  

**为什么值得看**：  
- 越狱防御是 LLM 安全的核心问题之一，这篇代表了一种“从内部表征出发”的防御路线，可以和基于 RLHF / 
审查规则的办法做对比。

---

### 2. TwinBreak: Jailbreaking LLM Security Alignments based on Twin Prompts  
**会议**：USENIX Security 2025  

**关键词**：越狱攻击、提示结构设计、对齐绕过。  
**可能内容**：
- 提出一种 “Twin Prompts（双提示）” 
结构，通过构造两段关联提示，让模型在遵守一段指令的同时无意间违反另一段更高层次的安全对齐约束。
- 系统化研究这种结构是否对主流对齐策略（RLHF、system prompt 等）具有普适威力。  

**为什么值得看**：  
- 如果你在研究越狱攻击方法，这篇可能给出了一套新的攻击范式与评测基准。  
- 也可作为防御方理解攻击模式的素材。

---

### 3. SecAlign: Defending Against Prompt Injection with Preference Optimization  
**会议**：CCS 2025  

**关键词**：Prompt Injection、防御、偏好优化、对齐训练。  
**可能内容**：
- 把“对抗 Prompt Injection”转化为一个偏好优化问题：模型应该偏好遵守系统指令/有权限的指令，而拒绝遵守外部恶意指令。
- 可能基于 DPO、IPO 或 RLHF 类方法，在训练中加入“安全偏好对比数据”（遵守合法提示 vs 遵守注入提示）。  

**为什么值得看**：  
- Prompt Injection 是 LLM+工具 / 
LLM+检索系统中的关键威胁，这篇直接从“对齐训练”的视角给出防御方案，非常贴合实际应用场景。  

---

## 二、安全对齐与微调阶段的对齐保持

### 4. Alleviating the Fear of Losing Alignment in LLM Fine-tuning  
**会议**：IEEE S&P (Oakland/SP) 2025  

**关键词**：对齐保持、微调安全、alignment forgetting。  
**可能内容**：
- 研究在 SFT / LoRA / 任务微调时，如何避免已有的安全对齐能力“被覆盖”或“被遗忘”（alignment forgetting）。
- 可能提出新的正则化、参数隔离或对齐约束技术，让模型在适应新任务时仍保持安全属性。  

**为什么值得看**：  
- 
现实中企业/研究者经常在基础模型上做二次微调，这篇非常切实际地解决“微调会不会把安全对齐搞丢”的问题，是工程落地时必读
的方向。

---

### 5. Refusal Is Not an Option: Unlearning Safety Alignment of Large Language Models  
**会议**：USENIX Security 2025  

**关键词**：对齐反转、unlearning、安全绕过。  
**可能内容**：
- 研究是否可以通过某些训练/攻击手段，让模型逐步“忘掉”其安全对齐能力，使其更易输出有害内容。
- 可能结合参数编辑、对抗微调或持续学习来分析“对齐可逆性”和“安全不可逆性”的界限。  

**为什么值得看**：  
- 从攻防两侧都极有价值：  
  - 攻击者视角：说明现有对齐是否能够被系统性卸除。  
  - 防御与治理视角：促使提出“难以反对齐”的机制，比如结构性约束或不可逆安全模块。

---

## 三、后门 / 插件 / 生态层面的 LLM 威胁

### 6. From Purity to Peril: Backdooring Merged Models From "Harmless" Benign Components  
**会议**：USENIX Security 2025  

**关键词**：模型合并、后门、权重组合、开源模型供应链。  
**可能内容**：
- 研究当我们用“模型合并”（merge 
models、模型融合）技术时，即便每个单独模型看起来是“干净”的，也可能在合并后激发潜在后门或隐藏行为。
- 对开源 LLM 社区非常现实：当前大量模型是通过 merge / checkpoint mixing 得来。  

**为什么值得看**：  
- 如果你关注 LLM 供应链安全 / 开源模型生态，这篇非常关键。  
- 提醒大家：安全属性在模型合并时不是简单线性叠加，可能出现新的脆弱性。

---

### 7. The Philosopher's Stone: Trojaning Plugins of Large Language Models  
**会议**：NDSS 2025  

**关键词**：插件、工具调用、Trojan / 后门、生态攻击。  
**可能内容**：
- 针对 LLM 
的“插件/工具”系统，研究如何在插件层面植入后门（例如特定输入触发危险行为或数据泄露），而基础模型本身看起来仍然“安全
对齐”。
- 分析 LLM+工具生态中的信任边界、权限模型和 Trojan 注入面。  

**为什么值得看**：  
- 随着 LLM Agent / tool-using LLM 成为主流，这篇几乎是必读的“插件层安全”论文，有助于你构建系统级安全威胁模型。

---

## 四、评测与特殊人群安全（Youth Safety）

### 8. YouthSafe: A Youth-Centric Safety Benchmark and Safeguard Model for Large Language Models  
**会议**：CCS 2025  

**关键词**：青少年安全、内容审核、基准测试、安全护栏。  
**可能内容**：
- 从“青少年用户”的视角，定义适宜/不适宜内容标准，构建一个针对 LLM 的安全 benchmark（题目、对话场景等）。
- 可能同时训练一个专门的“Safeguard Model”，作为过滤或协同审查模块，保护未成年人免受不良内容影响。  

**为什么值得看**：  
- LLM 在教育、陪伴等领域的应用越来越多，未成年人保护是监管和产品落地的核心问题。  
- 这篇可以给你一个“用户群体敏感维度”的安全评测思路，而不仅是通用 NSFW / violence / hate 分类。

---

## 五、LLM 作为攻击增强器的安全研究

### 9. EchoLLM: LLM-Augmented Acoustic Eavesdropping Attack on Bone Conduction Headphones with mmWave Radar  
**会议**：USENIX Security 2025  

**关键词**：侧信道攻击、声学窃听、LLM-augmented。  
**可能内容**：
- 利用 mmWave 雷达探测骨传导耳机的物理信号，再用 LLM 进行语音内容/语义恢复或文本推断，从而实现窃听攻击。
- LLM 在这里作为“增强解码与语义理解”的组件。  

**为什么值得看**：  
- 代表一大类趋势：**LLM 作为攻击增强器**，不是攻击对象本身，而是提升攻击链路中的推理/识别能力。  
- 如果你关心“LLM + 传统安全攻击”的结合，这是一个很好的案例。

---

## 六、如何利用这份“精选列表”

如果你要写一篇关于 **LLM 安全/攻防的综述**，可以大致按以下结构阅读和组织：

1. **对齐与微调阶段安全**  
   - *Alleviating the Fear of Losing Alignment in LLM Fine-tuning*  
   - *Refusal Is Not an Option: Unlearning Safety Alignment of LLMs*

2. **越狱与 Prompt 攻

[Step 3: Duration 20.83 seconds| Input tokens: 17,506 | Output tokens: 6,901]

'我从你给出的 NDSS / Oakland (SP) / USENIX Security / CCS 2025 论文列表里，挑出几篇和“大语言模型安全”直接相关、很值得关注的工作，按主题简单分组，方便你做调研或写 survey。下面都是你日志里已经出现的论文，并结合题目+常见研究方向做简要解读。\n\n---\n\n## 一、越狱攻击与越狱防御（Jailbreak & Prompt 攻击）\n\n### 1. JBShield: Defending Large Language Models from Jailbreak Attacks through Activated Concept Analysis and Manipulation  \n**会议**：USENIX Security 2025  \n\n**关键词**：越狱防御、内部激活/概念分析、安全对齐保持。  \n**可能内容**：\n- 利用“激活概念分析”（activated concept analysis）来理解模型内部表征哪些“危险概念”（如暴力、仇恨等），在检测到这些概念被触发时进行抑制或重写，从而阻止越狱提示成功。\n- 属于“模型内部机制层面”的防御，而不仅是简单规则过滤。  \n\n**为什么值得看**：  \n- 越狱防御是 LLM 安全的核心问题之一，这篇代表了一种“从内部表征出发”的防御路线，可以和基于 RLHF / 审查规则的办法做对比。\n\n---\n\n### 2. TwinBreak: Jailbreaking LLM Security Alignments based on Twin Prompts  \n**会议**：USENIX Security 2025  \n\n**关键词**：越狱攻击、提示结构设计、对齐绕过。  \n**可能内容**：\n- 提出一种 “Twin Prompts（双提示）” 结构，通过构造两段关联提示，让模型在遵守一段指令的同时无意间违反另一段更高层次的安全对齐约束。\n- 系统化研究这种结构是否对主流对齐策略（RLHF、system prompt 等）具有普适威力。  \n\n**为什么值得看**：  \n- 如果你在研究越狱攻击方法，这篇可能给出了一套新的攻击范式与评测基准。  \n- 也可作为防御方理解攻击模式的素材。\n\n---

### 方案说明：SmolAgent CodeAgent（结合 `code_agent.yaml` 的 prompt 设计细节）
- 核心循环：强制每步遵循 “Thought → Code → Observation”，并用 `{{code_block_opening_tag}}`/`{{code_block_closing_tag}}` 包裹代码。中间信息需用 `print` 显式输出，下一步作为 Observation 读取。最终答案必须调用 `final_answer` 工具收束。
- 规划前置：系统 prompt 要求先做 facts survey（已知/待查/待推导）再产出高层计划；若重试或中断，使用 update_plan 模板复盘并继续，保证稳态迭代。
- 工具与调用规范：可用工具（Python 函数）被注入环境，必须按显式参数名调用，禁止 dict 传参，禁止使用未定义变量，也禁止变量名与工具同名；对无结构化输出的工具，提示分步打印避免链式依赖。
- 安全与约束：限制可导入模块白名单；提醒不要链过多非结构化结果，必要时拆分步；保持会话状态，可多轮复用已导入模块和已定义变量。
- 角色定位：将代理设为“能用代码解决任意任务的专家助手”，鼓励用工具+代码完成 RAG 检索、加工与总结；prompt 示例覆盖算术、检索、多模态 QA 等，教会模型如何 Thought/Code/Observation 协同。
- 输出与流式：示例强调逐步打印中间结果、最后用 `final_answer` 汇总；若有 stream 开启，能边执行边输出。
### 优点
- 自带规划、分步执行与错误恢复模板，减少裸 toolcall 的样板与状态管理成本。
- Thought/Code/Observation 将思路与代码解耦，便于调试审计；跨步保留状态，适合多轮 RAG。
- 明确的调用规范（参数名、变量定义、模块白名单）降低调用错误，提升可重复性与安全性。
- 扩展性好：添加/替换工具即可扩展能力，主逻辑无需重写。
### 缺点
- Prompt 体积大、指令冗长，占用上下文并增加生成时延，相比直接 toolcall 成本更高。
- 控制粒度不及手写状态机，精确流式或严格路径需额外封装。
- 依赖工具实现与输出结构；非结构化结果仍需手动清洗，工具安全需自管。
- 简单任务时，完整 Thought/Code/Observation 纪律会带来不必要的步骤和延迟。